## 3k字总结生成脚本

In [8]:
import os
import requests
import openai
from dotenv import load_dotenv
from notion_client import Client
from pprint import pprint
import httpx
from urllib3.exceptions import InsecureRequestWarning
import json

httpx_client = httpx.Client()
# 传入notion client对象
notion = Client(auth=os.getenv("NOTION_TOKEN"), client=httpx_client)


# 从.env文件中导入账号(ACCOUNT)和密码(PASSWORD)
dotenv_path = "../../.env"
load_dotenv(dotenv_path)
NOTION_PAGEID = os.getenv("NOTION_PAGEID")

# 设置OpenAI的API Key
openai.api_key = os.getenv("V3_APIKEY")
openai.base_url = "https://api.gpt.ge/v1/"
openai.default_headers = {"x-foo": "true"}

# 忽略不安全请求的警告
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

print("init & import done")
print('PAGEID=' + NOTION_PAGEID)

init & import done
PAGEID=42054e42fa794c82b0ad35524a2f57d0


获取Notion中以Page格式存储的页面

In [9]:
page_block_list = notion.blocks.children.list(**{
    "block_id": NOTION_PAGEID # page_id can be used to search children blocks.
})

all_end_page_block_list = []

In [10]:
end_page_count = 0

# 第一级 - 月份目录页
level_1 = page_block_list["results"]
# 循环获取该级别下的所有页面下的名为end的子页面
for year in level_1:
    month_menu_block_list = notion.blocks.children.list(**{
        "block_id": year["id"]
    })
    
    # 第二级 - 特定周目录页
    week_menu = month_menu_block_list["results"]
    for week in week_menu:
        weekday_menu_block_list = notion.blocks.children.list(**{
            "block_id": week["id"]
        })
        day_block_list = weekday_menu_block_list["results"]
        for day in day_block_list:
            # 获取所有的标题为end的子页面
            # pprint(day_block_list)
            if day['has_children'] is True:
                if day['child_page']['title'] == 'end':
                    all_end_page_block_list.append(notion.blocks.children.list(**{
                        "block_id": day["id"]
                    }))
                    end_page_count += 1
pprint(all_end_page_block_list)

[{'block': {},
  'has_more': False,
  'next_cursor': None,
  'object': 'list',
  'request_id': 'f13a63af-4cde-42a0-97e2-86e7476760af',
  'results': [{'archived': False,
               'created_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                              'object': 'user'},
               'created_time': '2024-12-30T05:55:00.000Z',
               'has_children': False,
               'id': '16c16695-04b0-8044-b78c-e2556f47eaa7',
               'in_trash': False,
               'last_edited_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                                  'object': 'user'},
               'last_edited_time': '2024-12-30T05:55:00.000Z',
               'object': 'block',
               'paragraph': {'color': 'default',
                             'rich_text': [{'annotations': {'bold': False,
                                                            'code': False,
                                                            'color': 'default',
     

In [24]:
pprint(all_end_page_block_list[20])
# for pages in all_end_page_block_list:
#     BlockList = pages['results']
#     pprint(BlockList)

{'block': {},
 'has_more': False,
 'next_cursor': None,
 'object': 'list',
 'request_id': 'a100a00e-d95f-49b9-aad3-4df413adda4b',
 'results': [{'archived': False,
              'created_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                             'object': 'user'},
              'created_time': '2024-12-27T15:38:00.000Z',
              'has_children': True,
              'id': '16916695-04b0-80c2-92ca-c2c6993c3ba0',
              'in_trash': False,
              'last_edited_by': {'id': '180a057d-bd2a-43dc-8cfb-ee9f7e93ef08',
                                 'object': 'user'},
              'last_edited_time': '2024-12-27T15:38:00.000Z',
              'numbered_list_item': {'color': 'default',
                                     'rich_text': [{'annotations': {'bold': False,
                                                                    'code': False,
                                                                    'color': 'default',
                       

In [27]:
def search_child_block(id):
    """
    获取指定id的子块
    :param id: 块或者页面ID
    :return: 
    """
    resp = notion.blocks.children.list(**{
        "block_id": id # page_id can be used to search children blocks.
    })
    return resp["results"]

def get_page_properties(id, property_name):
    """
    获取指定id页面的对应属性
    :param id: 块或者页面ID
    :return: 
    """
    resp = notion.pages.properties.retrieve(**{
        "page_id": id,
        "property_id": "title"
    })
    return resp["results"][0][property_name]


def get_all_text_blocks(block_list, indent_level=0):
    """
    递归查找所有文本块，并拼接到md_context中
    :param block_list: 该页面下的所有块列表
    :param indent_level: 缩进级别
    :return: 拼接后的文本内容
    """
    week_count = 0
    text_content = ""
    for block in block_list:
        if "paragraph" in block and block["paragraph"]["rich_text"]:
            indent = " " * (indent_level * 4)
            text_content += indent + block["paragraph"]["rich_text"][0]["text"]["content"] + "\n"
            
        if block["has_children"] and block['type'] == 'numbered_list_item':
            text_content += search_text_blocks(block["id"], indent_level + 1)
            continue
    return text_content

def search_text_blocks(block_id, indent_level=0):
    """
    递归查找所有文本块，并拼接到md_context中
    :param block_id: 块或者页面ID
    :param indent_level: 缩进级别
    :return: 拼接后的文本内容
    """
    blocks = search_child_block(block_id)
    text_content = ""
    for block in blocks:
        if "paragraph" in block and block["paragraph"]["rich_text"]:
            indent = " " * (indent_level * 4)
            text_content += indent + block["paragraph"]["rich_text"][0]["text"]["content"] + "\n"
        if block["has_children"]:
            text_content += get_all_text_blocks(block["id"], indent_level + 1)
    return text_content

print("func init done")
md_context = ""
weekNum = 1

for pages in all_end_page_block_list:
    BlockList = pages['results']
    md_context += f"### 第 {weekNum} 周周记\n"
    md_context += get_all_text_blocks(BlockList)
    weekNum += 1
print("md_context gen done")

func init done
md_context gen done


In [28]:
pprint(md_context)

('### 第 1 周周记\n'
 '一、职位任务及完成情况\n'
 '二、心得与体会\n'
 '在本周的工作中，我深刻体会到有条不紊是做任何事情的核心要点。无论任务多么繁重或紧急，只要能够有计划地安排时间和资源，就能够有效地完成工作，保持高效的工作状态。\n'
 '三、存在问题与解决办法\n'
 '在与甲方的沟通过程中，遇到了一些问题。例如，甲方提出的具体需求是“五彩斑斓的黑”，这在实际操作中存在较大的挑战。为了解决这个问题，我制定了最小落地方案，并将该方案提供给甲方供其选择。通过这种方式，我们能够在一定程度上满足甲方的需求，同时确保工作的可行性和有效性。\n'
 '在接下来的工作中，我将继续保持与甲方的紧密沟通，努力推进未完成的任务，并持续优化我们的工作流程。\n'
 '### 第 2 周周记\n'
 '与行政部门及熊教授对接需求，完成对接并进入原型图绘制阶段。\n'
 '拉取“浏阳慈善pc管理端”代码，编写两个代码格式优化提交，并成功在本地对接测试服务器，为后续开发做准备。\n'
 '7月23日，2024年\n'
 '根据行政部门提供的原型图部署项目代码，完成基本配置，但尚未完成全部配置。\n'
 '等待教授回复以确认项目是否挂在公司名下，准备开发“生命之春”项目。\n'
 '7月24日，2024年\n'
 '基本部署项目代码，并与熊教授沟通项目进展。\n'
 '进一步将以前的项目Idea整合到毕设项目中，开始开发毕设项目。\n'
 '7月25日，2024年\n'
 '项目进行大规模修改，并将修改内容上传到Git。\n'
 '考虑程序依赖的Git同步问题，与熊教授和龙哥对接小程序前后端，上线demo版本。\n'
 '7月26日，2024年\n'
 '递交后端代码，并对前端页面进行初步测试与优化。\n'
 '与综合部对接原型图，准备正式开始项目页面开发。\n'
 '2. 心得与体会\n'
 '通过本周的工作，进一步提升了对项目代码的优化和配置能力。\n'
 '在与熊教授和团队的沟通中，学到了很多关于项目管理和前后端对接的经验。\n'
 '初步测试和优化前端页面，感受到了团队合作的重要性以及系统测试的重要性。\n'
 '3. 存在问题与解决办法\n'
 '问题：项目配置过程中遇到一些依赖问题，导致进度稍有延误。\n'
 '解决办法：计划与熊教授和龙哥进一步对接，确

In [29]:
rule = """
### 编写规则
在编写时，请注意以下几点：
1. **请务必不要少于3000字**
2. 请不要以“我们”为第一叙述人，而是以“我”为第一叙述人
3. 不要总是使用最日常的词语。在本例子中,以下词语的使用次数必须少于3次:独特的、确保、最大程度的。
4. 在输出内容之前,检查以下词语并用适当的替代词重写那些句子:细致的、仔细地、导航、复杂性、领域、定制的、量身定制的、朝着、支撑、不断变化的、不断发展的、…的世界、不仅、寻求不只是、旨在增强、不仅仅是、我们的套餐、建议、令人生畏的、在…的核心、当涉及到、在…的领域、在…当中、揭开秘密、揭示秘密、以及强大的。
5. 确保段落内容多样化，确保句子长度多样化。

请在创作以下标题为[实习总结]、字数为[3000]字的内容时坚持上述要求。
"""

prompt = "我在大学期间的实习期已经将近结束，我需要根据我在以前写的周报写一份实习总结，请根据我在下面编写的规则和所有周报，编写一篇实习总结：" + rule + md_context
# pprint(md_context)

# !IMPORTANT 请关闭系统代理再执行该部分代码
completion = openai.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
)

print("gen result text done!")

gen result text done!


In [30]:
# 将生成的总结保存到./result.md中
with open("./result.md", "w", encoding="utf-8") as f:
    f.write(completion.choices[0].message.content)
print("write to file done!")

write to file done!
